<a href="https://colab.research.google.com/github/syzygy21/robot_graphSLAMNavigation_andobstacleavoidance/blob/main/robot_SLAM_navigation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
from math import *
import random

In [2]:
class matrix:
    
    # implements basic operations of a matrix class

    # ------------
    #
    # initialization - can be called with an initial matrix
    #

    def __init__(self, value = [[]]):
        self.value = value
        self.dimx  = len(value)
        self.dimy  = len(value[0])
        if value == [[]]:
            self.dimx = 0

    # ------------
    #
    # makes matrix of a certain size and sets each element to zero
    #

    def zero(self, dimx, dimy):
        if dimy == 0:
            dimy = dimx
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            raise ValueError ("Invalid size of matrix")
        else:
            self.dimx  = dimx
            self.dimy  = dimy
            self.value = [[0.0 for row in range(dimy)] for col in range(dimx)]

    # ------------
    #
    # makes matrix of a certain (square) size and turns matrix into identity matrix
    #

    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
            raise ValueError ("Invalid size of matrix")
        else:
            self.dimx  = dim
            self.dimy  = dim
            self.value = [[0.0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1.0
    # ------------
    #
    # prints out values of matrix
    #

    def show(self, txt = ''):
        for i in range(len(self.value)):
            print (txt + '['+ ', '.join('%.3f'%x for x in self.value[i]) + ']') 
        print (' ')

    # ------------
    #
    # defines elmement-wise matrix addition. Both matrices must be of equal dimensions
    #

    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise ValueError ("Matrices must be of equal dimension to add")
        else:
            # add if correct dimensions
            res = matrix()
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res

    # ------------
    #
    # defines elmement-wise matrix subtraction. Both matrices must be of equal dimensions
    #

    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise ValueError ("Matrices must be of equal dimension to subtract")
        else:
            # subtract if correct dimensions
            res = matrix()
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res

    # ------------
    #
    # defines multiplication. Both matrices must be of fitting dimensions
    #

    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
            raise ValueError ("Matrices must be m*n and n*p to multiply")
        else:
            # multiply if correct dimensions
            res = matrix()
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
        return res


    # ------------
    #
    # returns a matrix transpose
    #

    def transpose(self):
        # compute transpose
        res = matrix()
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res

    # ------------
    #
    # creates a new matrix from the existing matrix elements.
    #
    # Example:
    #       l = matrix([[ 1,  2,  3,  4,  5], 
    #                   [ 6,  7,  8,  9, 10], 
    #                   [11, 12, 13, 14, 15]])
    #
    #       l.take([0, 2], [0, 2, 3])
    #
    # results in:
    #       
    #       [[1, 3, 4], 
    #        [11, 13, 14]]
    #       
    # 
    # take is used to remove rows and columns from existing matrices
    # list1/list2 define a sequence of rows/columns that shall be taken
    # if no list2 is provided, then list2 is set to list1 (good for 
    # symmetric matrices)
    #

    def take(self, list1, list2 = []):
        if list2 == []:
            list2 = list1
        if len(list1) > self.dimx or len(list2) > self.dimy:
            raise ValueError ("list invalid in take()")

        res = matrix()
        res.zero(len(list1), len(list2))
        for i in range(len(list1)):
            for j in range(len(list2)):
                res.value[i][j] = self.value[list1[i]][list2[j]]
        return res

    # ------------
    #
    # creates a new matrix from the existing matrix elements.
    #
    # Example:
    #       l = matrix([[1, 2, 3],
    #                  [4, 5, 6]])
    #
    #       l.expand(3, 5, [0, 2], [0, 2, 3])
    #
    # results in:
    #
    #       [[1, 0, 2, 3, 0], 
    #        [0, 0, 0, 0, 0], 
    #        [4, 0, 5, 6, 0]]
    # 
    # expand is used to introduce new rows and columns into an existing matrix
    # list1/list2 are the new indexes of row/columns in which the matrix
    # elements are being mapped. Elements for rows and columns 
    # that are not listed in list1/list2 
    # will be initialized by 0.0.
    #
    
    def expand(self, dimx, dimy, list1, list2 = []):
        if list2 == []:
            list2 = list1
        if len(list1) > self.dimx or len(list2) > self.dimy:
            raise ValueError ("list invalid in expand()")

        res = matrix()
        res.zero(dimx, dimy)
        for i in range(len(list1)):
            for j in range(len(list2)):
                res.value[list1[i]][list2[j]] = self.value[i][j]
        return res

    # ------------
    #
    # Computes the upper triangular Cholesky factorization of  
    # a positive definite matrix.
    # This code is based on http://adorio-research.org/wordpress/?p=4560
    #
    
    def Cholesky(self, ztol= 1.0e-5):

        res = matrix()
        res.zero(self.dimx, self.dimx)

        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else: 
                if d < 0.0:
                    raise ValueError ("Matrix not positive-definite")
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(i)])
                if abs(S) < ztol:
                    S = 0.0
                try:
                   res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
                except:
                   raise ValueError ("Zero diagonal")
        return res 
 
    # ------------
    #
    # Computes inverse of matrix given its Cholesky upper Triangular
    # decomposition of matrix.
    # This code is based on http://adorio-research.org/wordpress/?p=4560
    #
    
    def CholeskyInverse(self):

        res = matrix()
        res.zero(self.dimx, self.dimx)

        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/ tjj**2 - S/ tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = \
                    -sum([self.value[i][k]*res.value[k][j] for k in \
                              range(i+1,self.dimx)])/self.value[i][i]
        return res
    
    # ------------
    #
    # computes and returns the inverse of a square matrix
    #
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res

    # ------------
    #
    # prints matrix (needs work!)
    #
    def __repr__(self):
        return repr(self.value)

In [16]:
class robot:

    # --------
    # init: 
    #   creates robot and initializes location to 0, 0
    #

    def __init__(self, world_size = 100.0, measurement_range = 30.0,
                 motion_noise = 1.0, measurement_noise = 1.0):
        self.measurement_noise = 0.0
        self.world_size = world_size
        self.measurement_range = measurement_range
        self.x = world_size / 2
        self.y = world_size / 2
        self.motion_noise = motion_noise
        self.measurement_noise = measurement_noise
        self.landmarks = []
        self.num_landmarks = 0


    def rand(self):
        return random.random() * 2.0 - 1.0

    # --------
    #
    # make random landmarks located in the world
    #

    def make_landmarks(self, num_landmarks):
        self.landmarks = []
        for i in range(num_landmarks):
            self.landmarks.append([round(random.random() * self.world_size),
                                   round(random.random() * self.world_size)])
        self.num_landmarks = num_landmarks


    # --------
    #
    # move: attempts to move robot by dx, dy. If outside world
    #       boundary, then the move does nothing and instead returns failure
    #

    def move(self, dx, dy, obstacle_list):

        check = 0
        x = self.x + dx + self.rand() * self.motion_noise
        y = self.y + dy + self.rand() * self.motion_noise
         
        for i in range(len(obstacle_list)) :
            if (math.sqrt((x - obstacle_list[i][0]) ** 2 + (y - obstacle_list[i][1])**2) < 10) :
                check = 1

        if x < 0.0 or x > self.world_size or y < 0.0 or y > self.world_size or check == 1 :
            return False
        else:
            self.x = x
            self.y = y
            return True
    

    # --------
    #
    # sense: returns x- and y- distances to landmarks within visibility range
    #        because not all landmarks may be in this range, the list of measurements
    #        is of variable length. Set measurement_range to -1 if you want all
    #        landmarks to be visible at all times
    #

    def sense(self):
        Z = []
        for i in range(self.num_landmarks):
            dx = self.landmarks[i][0] - self.x + self.rand() * self.measurement_noise
            dy = self.landmarks[i][1] - self.y + self.rand() * self.measurement_noise    
            if self.measurement_range < 0.0 or abs(dx) + abs(dy) <= self.measurement_range:
                Z.append([i, dx, dy])
        return Z

    # --------
    #
    # print robot location
    #

    def __repr__(self):
        return 'Robot: [x=%.5f y=%.5f]'  % (self.x, self.y)

######################################################

# --------
# this routine makes the robot data
#



In [12]:
def make_data(N, r, num_landmarks, measurement_range, motion_noise, 
              measurement_noise,init_x, init_y, goal_x, goal_y,obstacle_list, distance):


    check = 0
    if math.sqrt((init_x - goal_x)**2 + (init_y - goal_y)**2) < 5: 
        check = 1
    complete = False
    while not complete:

        data = []

        # make robot and landmarks
        #r = robot(world_size, measurement_range, motion_noise, measurement_noise)
        #r.make_landmarks(num_landmarks)
        seen = [False for row in range(num_landmarks)]
    
        # guess an initial motion
        #orientation = random.random() * 2.0 * pi
        orientation = 3.14159265 - math.atan2(-(init_y - goal_y), init_x - goal_x)
        dx = cos(orientation) * distance
        dy = sin(orientation) * distance
    
        for k in range(N-1):
    
            # sense
            Z = r.sense()

            # check off all landmarks that were observed 
            for i in range(len(Z)):
                seen[Z[i][0]] = True
    
            # move
            while not r.move(dx, dy, obstacle_list):
                # if we'd be leaving the robot world, pick instead a new direction
                print("collision or out of world")
                orientation = random.random() * 2.0 * pi
                dx = cos(orientation) * distance
                dy = sin(orientation) * distance
            if check == 1 :
                break 
            # memorize data
            data.append([Z, [dx, dy]])

        # we are done when all landmarks were observed; otherwise re-run
        if check == 1 :
            break
        else :
            complete = (sum(seen) == num_landmarks)

    print (' ')
    print ('Landmarks: ', r.landmarks)
    print (robot)


    return data, check

In [6]:
def slam(data, N, num_landmarks, motion_noise, measurement_noise,init_x, init_y, world_size):    
    
    dim = 2 * (N + num_landmarks)
    
    Omega = matrix()
    Omega.zero(dim , dim)
    Omega.value[0][0] = 1.0
    Omega.value[1][1] = 1.0
    
    Xi = matrix()
    Xi.zero(dim, 1)
    Xi.value[0][0] = init_x
    Xi.value[1][0] = init_y
    
    for k in range(len(data)) :
        
        n = k * 2 
        
        measurement = data[k][0]
        motion = data[k][1]
        for i in range(len(measurement)) :
            
            m = 2 * (N + measurement[i][0])
            
            for b in range (2) :
                Omega.value[n+b][n+b] += 1.0 / measurement_noise
                Omega.value[m+b][m+b] += 1.0 / measurement_noise
                Omega.value[n+b][m+b] += -1.0 / measurement_noise
                Omega.value[m+b][n+b] += -1.0 / measurement_noise
                Xi.value[n+b][0] += -measurement[i][1+b] / measurement_noise
                Xi.value[m+b][0] += measurement[i][1+b] / measurement_noise
        for b in range (4) :
            Omega.value[n+b][n+b] += 1.0 / motion_noise 
        for b in range (2) :
            Omega.value[n+b][n+b+2] += -1.0 / motion_noise
            Omega.value[n+b+2][n+b] += -1.0 / motion_noise
            Xi.value[n+b][0] += -motion[b] / motion_noise
            Xi.value[n+b+2][0] += motion[b] / motion_noise
            
    mu = Omega.inverse() * Xi
    return (mu)

In [17]:
num_landmarks      = 5        # number of landmarks
N                  = 2       # time steps
world_size         = 100.0    # size of world
measurement_range  = -1       # range at which we can sense landmarks
motion_noise       = 2.0      # noise in robot motion
measurement_noise  = 2.0      # noise in the measurements
distance           = 5     # distance by which robot (intends to) move each iteratation 
obstacle_list = [[60, 50]]
init_x = 50
init_y = 50
goal_x = 70
goal_y = 50
r = robot(world_size, measurement_range, motion_noise, measurement_noise)
r.make_landmarks(num_landmarks)


while True :
    data, check = make_data(N, r, num_landmarks, measurement_range, motion_noise, measurement_noise,init_x, init_y, goal_x, goal_y, obstacle_list, distance)
    if(check == 1) :
        print("Reached")
        break
    result = slam(data, N, num_landmarks, motion_noise, measurement_noise,init_x, init_y, world_size)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
    print(result)
    print_result(N, num_landmarks, result) 
    init_y = result.value[2*N - 1][0]
    init_x = result.value[2*N - 2][0]
    
    #obstacle_list = [ ]

#init_x = 70
#init_y = 50
#goal_x = 50
#goal_y = 70


collision or out of world
collision or out of world
collision or out of world
 
Landmarks:  [[79, 4], [32, 55], [23, 48], [49, 35], [35, 56]]
<class '__main__.robot'>
[[50.0], [50.0], [48.87810498135792], [54.872509781123675], [79.27181457324238], [2.3545669528500723], [32.241285749640106], [55.00109898338817], [24.9379679336433], [46.532064230242206], [50.3597614453298], [36.9614035194388], [35.61960569785362], [54.50998234756201]]

Estimated Pose(s):
    [50.000, 50.000]
    [48.878, 54.873]

Estimated Landmarks:
    [79.272, 2.355]
    [32.241, 55.001]
    [24.938, 46.532]
    [50.360, 36.961]
    [35.620, 54.510]
collision or out of world
 
Landmarks:  [[79, 4], [32, 55], [23, 48], [49, 35], [35, 56]]
<class '__main__.robot'>
[[48.8781049813579], [54.872509781123675], [44.40639559644721], [57.109430695439066], [77.4761951357708], [4.070368007572129], [30.9910398082872], [54.04905121036271], [21.388190714038664], [50.29594229956638], [47.82891873765908], [37.51407159991026], [35.073

In [7]:
def print_result(N, num_landmarks, result):
    print ()
    print ('Estimated Pose(s):')
    for i in range(N):
        print ('    ['+ ', '.join('%.3f'%x for x in result.value[2*i]) + ', ' \
            + ', '.join('%.3f'%x for x in result.value[2*i+1]) +']')
    print ()
    print ('Estimated Landmarks:')
    for i in range(num_landmarks):
        print ('    ['+ ', '.join('%.3f'%x for x in result.value[2*(N+i)]) + ', ' \
            + ', '.join('%.3f'%x for x in result.value[2*(N+i)+1]) +']')

In [ ]:
init_x = 70
init_y = 50
goal_x = 90
goal_y = 70
print(  3.14159265 - math.atan2(-(init_y - goal_y), init_x - goal_x) )

0.7853981598076554


In [ ]:
math.sqrt((52.322 - 50.000)**2 + ( 67.678 - 70)**2)

3.2838038918303307